In [1]:
import torch
from torch import nn

from yolox.exp import get_exp
from yolox.models.network_blocks import SiLU
from yolox.utils import replace_module

import torch
from yolox.models import YOLOPAFPN_234, YOLOPAFPN

In [2]:
fpn = YOLOPAFPN(0.33, 0.50)
fpn2 = YOLOPAFPN_234(0.33, 0.50)

In [3]:
fpn = replace_module(fpn, nn.SiLU, SiLU)
fpn2 = replace_module(fpn2, nn.SiLU, SiLU)

In [4]:
def sim(path):
    import onnx
    from onnxsim import simplify

    onnx_model = onnx.load(path)
    model_simp, check = simplify(onnx_model)
    assert check, "Simplified ONNX model could not be validated"
    onnx.save(model_simp, path)

In [5]:
dummy_input = torch.randn(1, 3, 640, 640)

torch.onnx._export(
    fpn,
    dummy_input,
    "fpn.onnx",
    input_names=["images"],
    opset_version=11,
)
sim("fpn.onnx")

/home/zhanghao/.local/lib/python3.6/site-packages/torch/onnx/utils.py:46: UserWarning: You are exporting the model to ONNX while in training mode with 'train' parameter not specified. The model will default to inference mode export. If you wish to export a training amenable ONNX model, specify training=TrainingMode.TRAINING or training=TrainingMode.PRESERVE (to preserve the original model state) in torch.onnx.export().
  warnings.warn("You are exporting the model to ONNX while in training mode with "
/home/zhanghao/.local/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
torch.onnx._export(
    fpn2,
    dummy_input,
    "fpn2.onnx",
    input_names=["images"],
    opset_version=11,
)
sim("fpn2.onnx")

In [7]:
from yolox.models import CSPDarknet

backbone = CSPDarknet(0.33, 0.50)

In [8]:
dummy_input = torch.randn(1, 3, 640, 640)
torch.onnx._export(
    backbone,
    dummy_input,
    "backbone.onnx",
    input_names=["images"],
    opset_version=11,
)
sim("backbone.onnx")